In [ ]:
# Generate multi-source billing/invoice exports (synthetic) inside Colab ===
# Import libraries
import os, json
from pathlib import Path
import numpy as np
import pandas as pd

# Create folders to store raw exports and outputs
BASE = Path("/content/finops_showcase")
DATA = BASE / "data"
OUT  = BASE / "output"
DATA.mkdir(parents=True, exist_ok=True)
OUT.mkdir(parents=True, exist_ok=True)

# Fix randomness for reproducibility
rng = np.random.default_rng(42)

# Configure display for easier inspection
pd.set_option("display.max_columns", 140)

print("Folders ready:", BASE, DATA, OUT)


Folders ready: /content/finops_showcase /content/finops_showcase/data /content/finops_showcase/output


In [ ]:
# Define common dimensions to reuse across sources
owners = ["pedro", "ana", "diego", "sebastian", "claudio"]
apps   = ["sales-portal", "ai-agent-vt", "data-platform", "mobile-app", "erp-integration"]
envs   = ["dev", "test", "prod"]
ccs    = ["CC100-Sales", "CC200-Data", "CC300-Tech", "CC400-Finance"]
bus    = ["UEN-A", "UEN-B", "UEN-C"]

# Generate tags/labels and intentionally drop some keys to simulate untagged/poorly-tagged spend
def make_tags(n, missing_rate=0.18):
    tags = []
    for _ in range(n):
        t = {
            "owner": rng.choice(owners),
            "app": rng.choice(apps),
            "env": rng.choice(envs),
            "cost_center": rng.choice(ccs),
            "business_unit": rng.choice(bus),
        }
        # Drop random tags to create allocation gaps to fix later
        for k in list(t.keys()):
            if rng.random() < missing_rate:
                t.pop(k, None)
        tags.append(t)
    return tags


In [ ]:
# Create a 30-day date spine to simulate daily line items
days = pd.date_range(end=pd.Timestamp.today().normalize(), periods=30, freq="D")

# Define shared “service-like” and “sku-like” fields to support normalization later
regions  = ["us-east-1", "us-west-2", "sa-east-1", "europe-west1", "eastus", "westeurope"]
services = ["Compute", "Storage", "Database", "Network", "AI/ML", "Kubernetes"]
skus     = ["standard", "premium", "gpu", "archive", "serverless", "managed"]

# Round floats in a consistent way for money/usage columns
def money(x):
    return np.round(np.asarray(x, dtype=float), 4)


In [ ]:
# Create an AWS CUR-like export (simplified but with realistic “CUR flavor” columns)
n = 2500
aws = pd.DataFrame({
    "lineItem/UsageStartDate": rng.choice(days, n),
    "bill/PayerAccountId": rng.integers(100000000000, 999999999999, n).astype(str),
    "lineItem/UsageAccountId": rng.integers(100000000000, 999999999999, n).astype(str),
    "product/ProductName": rng.choice(services, n),
    "lineItem/ResourceId": ["arn:aws:res/" + str(rng.integers(10**8, 10**9)) for _ in range(n)],
    "product/region": rng.choice(regions, n),
    "lineItem/UsageType": rng.choice(["BoxUsage", "DataTransfer", "Requests", "GB-Mo", "vCPU-Hours"], n),
    "lineItem/UsageAmount": money(rng.gamma(2.0, 3.0, n)),
    "pricing/unit": rng.choice(["Hrs", "GB", "Requests", "GB-Mo"], n),
    "lineItem/UnblendedRate": money(rng.uniform(0.01, 1.2, n)),
})

# Compute base cost and an amortized view to simulate commitments (RI/SP)
aws["lineItem/UnblendedCost"] = money(aws["lineItem/UsageAmount"] * aws["lineItem/UnblendedRate"])
aws["commitment/Type"] = rng.choice(["None", "SavingsPlan", "ReservedInstance"], n, p=[0.72, 0.18, 0.10])
aws["savings/Discount"] = money(np.where(aws["commitment/Type"].eq("None"), 0, rng.uniform(0.05, 0.55, n)))
aws["lineItem/AmortizedCost"] = money(aws["lineItem/UnblendedCost"] * (1 - aws["savings/Discount"]))

# Attach tags as JSON to simulate resource tags
aws["resourceTags/user:tags_json"] = [json.dumps(t) for t in make_tags(n)]

# Persist as a raw export file
aws.to_csv(DATA / "aws_cur_export.csv", index=False)

# Preview a few rows to confirm structure
aws.head(3)


,lineItem/UsageStartDate,bill/PayerAccountId,lineItem/UsageAccountId,product/ProductName,lineItem/ResourceId,product/region,lineItem/UsageType,lineItem/UsageAmount,pricing/unit,lineItem/UnblendedRate,lineItem/UnblendedCost,commitment/Type,savings/Discount,lineItem/AmortizedCost,resourceTags/user:tags_json
0,2025-11-20,189847951459,176372599199,Storage,arn:aws:res/167554617,eastus,GB-Mo,7.0296,GB-Mo,0.9069,6.3751,None,0.0,6.3751,"{""owner"": ""claudio"", ""app"": ""mobile-app"", ""env..."
1,2025-12-11,196662660659,560378022973,AI/ML,arn:aws:res/551386820,sa-east-1,DataTransfer,2.3991,Requests,1.1119,2.6676,None,0.0,2.6676,"{""owner"": ""diego"", ""app"": ""erp-integration"", ""..."
2,2025-12-07,203793794299,505663829036,Storage,arn:aws:res/612920837,us-east-1,DataTransfer,0.3539,GB,0.9071,0.3210,None,0.0,0.3210,"{""owner"": ""diego"", ""app"": ""ai-agent-vt"", ""cost..."


In [ ]:
# Create an Azure Cost Management-like export
n = 2300
azure = pd.DataFrame({
    "Date": rng.choice(days, n),
    "SubscriptionId": [f"sub-{rng.integers(10**7, 10**8)}" for _ in range(n)],
    "ResourceId": [f"/subscriptions/.../resourceGroups/rg{rng.integers(1,30)}/providers/Microsoft.X/{rng.integers(10**7,10**8)}" for _ in range(n)],
    "ServiceName": rng.choice(services, n),
    "MeterCategory": rng.choice(["Compute", "Storage", "Networking", "Database", "AI"], n),
    "Region": rng.choice(["eastus", "westeurope", "brazilsouth"], n),
    "Quantity": money(rng.gamma(2.2, 2.6, n)),
    "UnitOfMeasure": rng.choice(["Hours", "GB", "1K", "GB-Month"], n),
    "UnitPrice": money(rng.uniform(0.01, 1.1, n)),
})

# Compute cost fields and simulate discounted effective cost via reservations/savings plans
azure["CostInBillingCurrency"] = money(azure["Quantity"] * azure["UnitPrice"])
azure["PricingModel"] = rng.choice(["OnDemand", "Reservation", "SavingsPlan"], n, p=[0.74, 0.16, 0.10])
azure["EffectiveCost"] = money(np.where(
    azure["PricingModel"].eq("OnDemand"),
    azure["CostInBillingCurrency"],
    azure["CostInBillingCurrency"] * rng.uniform(0.55, 0.9, n)
))

# Attach tags as JSON
azure["Tags"] = [json.dumps(t) for t in make_tags(n)]

# Persist as a raw export file
azure.to_csv(DATA / "azure_cost_export.csv", index=False)

# Preview a few rows to confirm structure
azure.head(3)


,Date,SubscriptionId,ResourceId,ServiceName,MeterCategory,Region,Quantity,UnitOfMeasure,UnitPrice,CostInBillingCurrency,PricingModel,EffectiveCost,Tags
0,2025-11-29,sub-58462532,/subscriptions/.../resourceGroups/rg27/provide...,AI/ML,Database,eastus,3.3703,GB-Month,0.2748,0.9262,OnDemand,0.9262,"{""owner"": ""ana"", ""app"": ""ai-agent-vt"", ""env"": ..."
1,2025-12-17,sub-43406334,/subscriptions/.../resourceGroups/rg4/provider...,Storage,Networking,eastus,3.2694,GB,0.6955,2.2739,OnDemand,2.2739,"{""owner"": ""diego"", ""env"": ""test"", ""cost_center..."
2,2025-12-04,sub-89598558,/subscriptions/.../resourceGroups/rg10/provide...,Kubernetes,Database,westeurope,13.5570,Hours,0.7085,9.6051,OnDemand,9.6051,"{""owner"": ""ana"", ""app"": ""mobile-app"", ""cost_ce..."


In [ ]:
# Create a GCP Billing export-like dataset
n = 2400
gcp = pd.DataFrame({
    "usage_start_time": rng.choice(days, n),
    "billing_account_id": [f"{rng.integers(10**8,10**9)}-{rng.integers(10**4,10**5)}" for _ in range(n)],
    "project.id": [f"proj-{rng.integers(1000,9999)}" for _ in range(n)],
    "service.description": rng.choice(services, n),
    "sku.description": rng.choice(skus, n),
    "location.region": rng.choice(["us-central1", "europe-west1", "southamerica-east1"], n),
    "usage.amount": money(rng.gamma(2.1, 2.8, n)),
    "usage.unit": rng.choice(["hour", "gibibyte", "request", "gibibyte month"], n),
    "cost": money(rng.uniform(0.01, 1.2, n) * rng.gamma(2.0, 2.0, n)),
})

# Simulate credits/discounts and compute net cost
gcp["credits.amount"] = money(-np.where(rng.random(n) < 0.28, gcp["cost"] * rng.uniform(0.05, 0.5, n), 0))
gcp["net_cost"] = money(gcp["cost"] + gcp["credits.amount"])

# Attach labels as JSON
gcp["labels_json"] = [json.dumps(t) for t in make_tags(n)]

# Persist as a raw export file
gcp.to_csv(DATA / "gcp_billing_export.csv", index=False)

# Preview a few rows to confirm structure
gcp.head(3)


,usage_start_time,billing_account_id,project.id,service.description,sku.description,location.region,usage.amount,usage.unit,cost,credits.amount,net_cost,labels_json
0,2025-11-20,456758250-74900,proj-6984,Storage,premium,southamerica-east1,5.4369,gibibyte month,1.9352,-0.0,1.9352,"{""owner"": ""claudio"", ""app"": ""ai-agent-vt"", ""co..."
1,2025-12-09,654957521-38811,proj-3932,Database,archive,us-central1,14.4980,request,0.0546,-0.0,0.0546,"{""owner"": ""ana"", ""app"": ""sales-portal"", ""env"":..."
2,2025-12-09,966845336-79045,proj-5916,Kubernetes,archive,us-central1,8.3857,gibibyte,7.7853,-0.0,7.7853,"{""owner"": ""pedro"", ""app"": ""mobile-app"", ""env"":..."


In [ ]:
# Create an OCI cost export-like dataset
n = 1800
oci = pd.DataFrame({
    "usageDate": rng.choice(days, n),
    "tenancyId": [f"ocid1.tenancy.oc1..{rng.integers(10**12,10**13)}" for _ in range(n)],
    "compartmentName": rng.choice(["prod", "shared", "data", "innovation"], n),
    "service": rng.choice(services, n),
    "sku": rng.choice(skus, n),
    "region": rng.choice(["sa-santiago-1", "us-ashburn-1", "eu-frankfurt-1"], n),
    "usageQuantity": money(rng.gamma(1.9, 3.1, n)),
    "unit": rng.choice(["OCPU-Hours", "GB", "Requests", "GB-Month"], n),
    "unitPrice": money(rng.uniform(0.01, 0.95, n)),
})

# Compute cost and simulate discounts to obtain net cost
oci["cost"] = money(oci["usageQuantity"] * oci["unitPrice"])
oci["discount"] = money(np.where(rng.random(n) < 0.20, rng.uniform(0.05, 0.35, n), 0))
oci["netCost"] = money(oci["cost"] * (1 - oci["discount"]))

# Attach freeform tags as JSON
oci["freeformTags"] = [json.dumps(t) for t in make_tags(n)]

# Persist as a raw export file
oci.to_csv(DATA / "oci_cost_export.csv", index=False)

# Preview a few rows to confirm structure
oci.head(3)


,usageDate,tenancyId,compartmentName,service,sku,region,usageQuantity,unit,unitPrice,cost,discount,netCost,freeformTags
0,2025-12-02,ocid1.tenancy.oc1..9803537327390,prod,Database,premium,sa-santiago-1,13.2712,Requests,0.2470,3.2780,0.0,3.2780,"{""owner"": ""sebastian"", ""app"": ""mobile-app"", ""e..."
1,2025-11-30,ocid1.tenancy.oc1..4713931077008,shared,Compute,gpu,eu-frankfurt-1,6.8135,GB-Month,0.6374,4.3429,0.0,4.3429,"{""owner"": ""claudio"", ""app"": ""data-platform"", ""..."
2,2025-11-21,ocid1.tenancy.oc1..6272439074015,prod,AI/ML,gpu,eu-frankfurt-1,2.1181,GB-Month,0.5727,1.2130,0.0,1.2130,"{""owner"": ""diego"", ""app"": ""mobile-app"", ""busin..."


In [ ]:
# Create a Kubernetes / Kubecost-like export to support namespace/workload allocation
n = 1200
k8s = pd.DataFrame({
    "date": rng.choice(days, n),
    "cluster": rng.choice(["gke-prod", "eks-shared", "aks-dev"], n),
    "namespace": rng.choice(["sales", "data", "platform", "ai", "monitoring"], n),
    "workload": rng.choice(["deployment/api", "deployment/worker", "statefulset/db", "job/etl"], n),
    "cpu_core_hours": money(rng.gamma(2.0, 1.6, n)),
    "mem_gb_hours": money(rng.gamma(2.0, 2.0, n)),
})

# Convert resource consumption into cost (simple model; refine later if desired)
k8s["cost"] = money(
    k8s["cpu_core_hours"] * rng.uniform(0.02, 0.08, n) +
    k8s["mem_gb_hours"]  * rng.uniform(0.001, 0.01, n)
)

# Attach labels as JSON (use higher missing rate to force allocation work later)
k8s["labels_json"] = [json.dumps(t) for t in make_tags(n, missing_rate=0.25)]

# Persist as a raw export file
k8s.to_csv(DATA / "k8s_kubecost_export.csv", index=False)

# Preview a few rows to confirm structure
k8s.head(3)


,date,cluster,namespace,workload,cpu_core_hours,mem_gb_hours,cost,labels_json
0,2025-12-13,eks-shared,platform,deployment/worker,5.7314,10.8191,0.4370,"{""owner"": ""diego"", ""app"": ""erp-integration"", ""..."
1,2025-12-13,eks-shared,monitoring,deployment/api,1.6472,1.5738,0.0642,"{""app"": ""erp-integration"", ""env"": ""test"", ""cos..."
2,2025-11-23,aks-dev,sales,deployment/api,1.9837,2.9754,0.1081,"{""owner"": ""diego"", ""app"": ""erp-integration"", ""..."


In [ ]:
# Create SaaS invoice exports (simulate Datadog/Snowflake/Atlassian/GitHub/etc.)
n = 180
saas = pd.DataFrame({
    "invoice_month": pd.to_datetime(rng.choice(pd.date_range(days.min(), days.max(), freq="MS"), n)),
    "vendor": rng.choice(["Datadog", "Snowflake", "Atlassian", "GitHub", "OpenAI", "Slack"], n),
    "product": rng.choice(["APM", "Logs", "Warehouse", "Seats", "API Usage"], n),
    "metric": rng.choice(["seats", "gb_ingested", "credits", "requests"], n),
    "quantity": money(rng.gamma(2.0, 25.0, n)),
    "unit_price": money(rng.uniform(0.02, 30.0, n)),
})

# Compute invoice cost and attach tags (with more gaps to force governance/cleanup later)
saas["cost"] = money(saas["quantity"] * saas["unit_price"])
saas["tags_json"] = [json.dumps(t) for t in make_tags(n, missing_rate=0.30)]

# Persist as a raw export file
saas.to_csv(DATA / "saas_invoices_export.csv", index=False)

# Preview a few rows to confirm structure
saas.head(3)


,invoice_month,vendor,product,metric,quantity,unit_price,cost,tags_json
0,2025-12-01,Slack,APM,gb_ingested,87.7546,9.7467,855.3178,"{""owner"": ""pedro"", ""app"": ""data-platform"", ""en..."
1,2025-12-01,Datadog,APM,requests,5.1768,28.3817,146.9264,"{""app"": ""mobile-app"", ""cost_center"": ""CC100-Sa..."
2,2025-12-01,Datadog,Logs,seats,108.1756,4.0277,435.6989,"{""owner"": ""sebastian"", ""env"": ""prod"", ""cost_ce..."


In [ ]:
# List generated files to confirm that all sources exist on disk
sorted([p.name for p in DATA.glob("*.csv")])


['aws_cur_export.csv',
 'azure_cost_export.csv',
 'gcp_billing_export.csv',
 'k8s_kubecost_export.csv',
 'oci_cost_export.csv',
 'saas_invoices_export.csv']

In [ ]:
# Loading each raw export from disk, mapping fields into one common line-item schema, unpacking tags/labels, and saving a single normalized dataset for later allocation + KPIs.
import json
import pandas as pd
from pathlib import Path

BASE = Path("/content/finops_showcase")
DATA = BASE / "data"
OUT  = BASE / "output"
OUT.mkdir(parents=True, exist_ok=True)

def _read_csv(name: str) -> pd.DataFrame:
    path = DATA / name
    if not path.exists():
        raise FileNotFoundError(f"Missing file: {path} (run Step 1 first)")
    return pd.read_csv(path)

def _unpack_tags(series: pd.Series) -> pd.DataFrame:
    def safe_load(x):
        try:
            return json.loads(x) if isinstance(x, str) and x.strip() else {}
        except Exception:
            return {}
    dicts = series.apply(safe_load)
    return pd.json_normalize(dicts).rename(columns={
        "owner":"tag_owner",
        "app":"tag_app",
        "env":"tag_env",
        "cost_center":"tag_cost_center",
        "business_unit":"tag_business_unit",
    })

COMMON_COLS = [
    "date","source",
    "account_id","project_id",
    "service","sku","region","resource_id",
    "usage_amount","usage_unit",
    "cost","net_cost","amortized_cost",
    "commitment_type",
    "tag_owner","tag_app","tag_env","tag_cost_center","tag_business_unit",
    "raw_tags_json"
]

# Normalizing AWS CUR
aws = _read_csv("aws_cur_export.csv")
aws_tags = _unpack_tags(aws["resourceTags/user:tags_json"])
aws_norm = pd.DataFrame({
    "date": pd.to_datetime(aws["lineItem/UsageStartDate"]).dt.date,
    "source": "aws",
    "account_id": aws["lineItem/UsageAccountId"],
    "project_id": None,
    "service": aws["product/ProductName"],
    "sku": aws["lineItem/UsageType"],
    "region": aws["product/region"],
    "resource_id": aws["lineItem/ResourceId"],
    "usage_amount": aws["lineItem/UsageAmount"],
    "usage_unit": aws["pricing/unit"],
    "cost": aws["lineItem/UnblendedCost"],
    "net_cost": aws["lineItem/UnblendedCost"],
    "amortized_cost": aws.get("lineItem/AmortizedCost", aws["lineItem/UnblendedCost"]),
    "commitment_type": aws.get("commitment/Type", "None"),
    "raw_tags_json": aws["resourceTags/user:tags_json"],
})
aws_norm = pd.concat([aws_norm, aws_tags], axis=1)

# Normalizing Azure Cost Management export
az = _read_csv("azure_cost_export.csv")
az_tags = _unpack_tags(az["Tags"])
az_norm = pd.DataFrame({
    "date": pd.to_datetime(az["Date"]).dt.date,
    "source": "azure",
    "account_id": az["SubscriptionId"],
    "project_id": None,
    "service": az["ServiceName"],
    "sku": az["MeterCategory"],
    "region": az["Region"],
    "resource_id": az["ResourceId"],
    "usage_amount": az["Quantity"],
    "usage_unit": az["UnitOfMeasure"],
    "cost": az["CostInBillingCurrency"],
    "net_cost": az.get("EffectiveCost", az["CostInBillingCurrency"]),
    "amortized_cost": az.get("EffectiveCost", az["CostInBillingCurrency"]),
    "commitment_type": az.get("PricingModel", "OnDemand"),
    "raw_tags_json": az["Tags"],
})
az_norm = pd.concat([az_norm, az_tags], axis=1)

# Normalizing GCP Billing export
gcp = _read_csv("gcp_billing_export.csv")
gcp_tags = _unpack_tags(gcp["labels_json"])
gcp_norm = pd.DataFrame({
    "date": pd.to_datetime(gcp["usage_start_time"]).dt.date,
    "source": "gcp",
    "account_id": gcp["billing_account_id"],
    "project_id": gcp["project.id"],
    "service": gcp["service.description"],
    "sku": gcp["sku.description"],
    "region": gcp["location.region"],
    "resource_id": None,
    "usage_amount": gcp["usage.amount"],
    "usage_unit": gcp["usage.unit"],
    "cost": gcp["cost"],
    "net_cost": gcp.get("net_cost", gcp["cost"]),
    "amortized_cost": gcp.get("net_cost", gcp["cost"]),
    "commitment_type": "None",
    "raw_tags_json": gcp["labels_json"],
})
gcp_norm = pd.concat([gcp_norm, gcp_tags], axis=1)

# Normalizing OCI cost export
oci = _read_csv("oci_cost_export.csv")
oci_tags = _unpack_tags(oci["freeformTags"])
oci_norm = pd.DataFrame({
    "date": pd.to_datetime(oci["usageDate"]).dt.date,
    "source": "oci",
    "account_id": oci["tenancyId"],
    "project_id": oci["compartmentName"],
    "service": oci["service"],
    "sku": oci["sku"],
    "region": oci["region"],
    "resource_id": None,
    "usage_amount": oci["usageQuantity"],
    "usage_unit": oci["unit"],
    "cost": oci["cost"],
    "net_cost": oci.get("netCost", oci["cost"]),
    "amortized_cost": oci.get("netCost", oci["cost"]),
    "commitment_type": "None",
    "raw_tags_json": oci["freeformTags"],
})
oci_norm = pd.concat([oci_norm, oci_tags], axis=1)

# Normalizing Kubernetes/Kubecost export
k8s = _read_csv("k8s_kubecost_export.csv")
k8s_tags = _unpack_tags(k8s["labels_json"])
k8s_norm = pd.DataFrame({
    "date": pd.to_datetime(k8s["date"]).dt.date,
    "source": "k8s",
    "account_id": k8s["cluster"],
    "project_id": k8s["namespace"],
    "service": "Kubernetes",
    "sku": k8s["workload"],
    "region": k8s["cluster"],
    "resource_id": k8s["workload"],
    "usage_amount": None,
    "usage_unit": None,
    "cost": k8s["cost"],
    "net_cost": k8s["cost"],
    "amortized_cost": k8s["cost"],
    "commitment_type": "None",
    "raw_tags_json": k8s["labels_json"],
})
k8s_norm = pd.concat([k8s_norm, k8s_tags], axis=1)

# Normalizing SaaS invoices export
saas = _read_csv("saas_invoices_export.csv")
saas_tags = _unpack_tags(saas["tags_json"])
saas_norm = pd.DataFrame({
    "date": pd.to_datetime(saas["invoice_month"]).dt.date,
    "source": "saas",
    "account_id": saas["vendor"],
    "project_id": None,
    "service": saas["vendor"],
    "sku": saas["product"],
    "region": None,
    "resource_id": saas["product"],
    "usage_amount": saas["quantity"],
    "usage_unit": saas["metric"],
    "cost": saas["cost"],
    "net_cost": saas["cost"],
    "amortized_cost": saas["cost"],
    "commitment_type": "Subscription",
    "raw_tags_json": saas["tags_json"],
})
saas_norm = pd.concat([saas_norm, saas_tags], axis=1)

# Stacking everything into one table and enforcing the final column order
finops_line_items = pd.concat([aws_norm, az_norm, gcp_norm, oci_norm, k8s_norm, saas_norm], ignore_index=True)
for c in COMMON_COLS:
    if c not in finops_line_items.columns:
        finops_line_items[c] = None
finops_line_items = finops_line_items[COMMON_COLS]

# Saving the normalized dataset for downstream allocation, KPIs, anomalies, and recommendations
finops_line_items.to_csv(OUT / "finops_line_items.csv", index=False)
try:
    finops_line_items.to_parquet(OUT / "finops_line_items.parquet", index=False)
except Exception as e:
    print("Parquet not saved (ok):", e)

print("Normalized rows:", len(finops_line_items), "| sources:", finops_line_items["source"].value_counts().to_dict())
finops_line_items.head(5)


/tmp/ipython-input-3842416113.py:176: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finops_line_items = pd.concat([aws_norm, az_norm, gcp_norm, oci_norm, k8s_norm, saas_norm], ignore_index=True)


Parquet not saved (ok): ("Could not convert 'sub-58462532' with type str: tried to convert to int64", 'Conversion failed for column account_id with type object')
Normalized rows: 10380 | sources: {'aws': 2500, 'gcp': 2400, 'azure': 2300, 'oci': 1800, 'k8s': 1200, 'saas': 180}


,date,source,account_id,project_id,service,sku,region,resource_id,usage_amount,usage_unit,cost,net_cost,amortized_cost,commitment_type,tag_owner,tag_app,tag_env,tag_cost_center,tag_business_unit,raw_tags_json
0,2025-11-20,aws,176372599199,None,Storage,GB-Mo,eastus,arn:aws:res/167554617,7.0296,GB-Mo,6.3751,6.3751,6.3751,NaN,claudio,mobile-app,test,CC400-Finance,UEN-B,"{""owner"": ""claudio"", ""app"": ""mobile-app"", ""env..."
1,2025-12-11,aws,560378022973,None,AI/ML,DataTransfer,sa-east-1,arn:aws:res/551386820,2.3991,Requests,2.6676,2.6676,2.6676,NaN,diego,erp-integration,test,CC400-Finance,UEN-B,"{""owner"": ""diego"", ""app"": ""erp-integration"", ""..."
2,2025-12-07,aws,505663829036,None,Storage,DataTransfer,us-east-1,arn:aws:res/612920837,0.3539,GB,0.3210,0.3210,0.3210,NaN,diego,ai-agent-vt,NaN,CC100-Sales,UEN-B,"{""owner"": ""diego"", ""app"": ""ai-agent-vt"", ""cost..."
3,2025-12-01,aws,776180930142,None,Storage,DataTransfer,sa-east-1,arn:aws:res/759155889,3.7181,GB,3.2768,3.2768,3.2768,NaN,ana,ai-agent-vt,dev,NaN,UEN-A,"{""owner"": ""ana"", ""app"": ""ai-agent-vt"", ""env"": ..."
4,2025-11-30,aws,518142328473,None,Database,Requests,eastus,arn:aws:res/269402834,3.1550,Hrs,0.9844,0.9844,0.9844,NaN,diego,sales-portal,test,CC300-Tech,UEN-C,"{""owner"": ""diego"", ""app"": ""sales-portal"", ""env..."


In [ ]:
# Load unified line-items table, measure tag completeness, apply allocation rules to fill missing tags, and save an allocation-ready dataset + coverage summary.
import pandas as pd
import numpy as np
from pathlib import Path

BASE = Path("/content/finops_showcase")
OUT  = BASE / "output"

df = pd.read_csv(OUT / "finops_line_items.csv", parse_dates=["date"])

# Define required tags for showback/chargeback allocation
required = ["tag_owner","tag_app","tag_env","tag_cost_center","tag_business_unit"]

# Define deterministic “best-effort” inference rules per source
ns_owner = {"sales":"ana","data":"diego","platform":"claudio","ai":"pedro","monitoring":"sebastian"}
ns_app   = {"sales":"sales-portal","data":"data-platform","platform":"erp-integration","ai":"ai-agent-vt","monitoring":"data-platform"}
ns_cc    = {"sales":"CC100-Sales","data":"CC200-Data","platform":"CC300-Tech","ai":"CC300-Tech","monitoring":"CC300-Tech"}
ns_bu    = {"sales":"UEN-A","data":"UEN-B","platform":"UEN-C","ai":"UEN-A","monitoring":"UEN-C"}

oci_owner = {"prod":"pedro","shared":"claudio","data":"diego","innovation":"ana"}
oci_env   = {"prod":"prod","shared":"test","data":"prod","innovation":"dev"}

saas_owner = {"Datadog":"claudio","Snowflake":"diego","Atlassian":"ana","GitHub":"claudio","OpenAI":"pedro","Slack":"ana"}
saas_app   = {"Datadog":"data-platform","Snowflake":"data-platform","Atlassian":"sales-portal","GitHub":"erp-integration","OpenAI":"ai-agent-vt","Slack":"sales-portal"}

# Create stable mappings for GCP projects so the same project infers the same values every run
proj_codes = pd.Series(df.loc[df["source"].eq("gcp"), "project_id"].fillna("proj-unknown").unique())
owners = ["pedro","ana","diego","sebastian","claudio"]
apps   = ["sales-portal","ai-agent-vt","data-platform","mobile-app","erp-integration"]
envs   = ["dev","test","prod"]
ccs    = ["CC100-Sales","CC200-Data","CC300-Tech","CC400-Finance"]
bus    = ["UEN-A","UEN-B","UEN-C"]

proj_map = {}
for i, p in enumerate(proj_codes):
    proj_map[p] = {
        "owner": owners[i % len(owners)],
        "app": apps[i % len(apps)],
        "env": envs[i % len(envs)],
        "cost_center": ccs[i % len(ccs)],
        "business_unit": bus[i % len(bus)],
    }

# Build allocation method tracker
df["alloc_method"] = "tagged"

def fill_alloc(col, values, method_name):
    m = df[col].isna() | (df[col].astype(str).str.strip() == "") | (df[col].astype(str).str.lower() == "nan")
    df.loc[m, col] = values[m]
    df.loc[m, "alloc_method"] = np.where(df.loc[m, "alloc_method"].eq("tagged"), method_name, df.loc[m, "alloc_method"])

# Infer from Kubernetes namespace when tags are missing
is_k8s = df["source"].eq("k8s")
ns = df["project_id"].fillna("").astype(str)
fill_alloc("tag_owner", ns.map(ns_owner).where(is_k8s), "inferred_k8s")
fill_alloc("tag_app", ns.map(ns_app).where(is_k8s), "inferred_k8s")
fill_alloc("tag_env", np.where(is_k8s & ns.eq("monitoring"), "prod", np.where(is_k8s, "prod", None)), "inferred_k8s")
fill_alloc("tag_cost_center", ns.map(ns_cc).where(is_k8s), "inferred_k8s")
fill_alloc("tag_business_unit", ns.map(ns_bu).where(is_k8s), "inferred_k8s")

# Infer from GCP project_id when tags are missing
is_gcp = df["source"].eq("gcp")
pid = df["project_id"].fillna("proj-unknown").astype(str)
fill_alloc("tag_owner", pid.map(lambda x: proj_map.get(x, {}).get("owner")).where(is_gcp), "inferred_gcp")
fill_alloc("tag_app", pid.map(lambda x: proj_map.get(x, {}).get("app")).where(is_gcp), "inferred_gcp")
fill_alloc("tag_env", pid.map(lambda x: proj_map.get(x, {}).get("env")).where(is_gcp), "inferred_gcp")
fill_alloc("tag_cost_center", pid.map(lambda x: proj_map.get(x, {}).get("cost_center")).where(is_gcp), "inferred_gcp")
fill_alloc("tag_business_unit", pid.map(lambda x: proj_map.get(x, {}).get("business_unit")).where(is_gcp), "inferred_gcp")

# Infer from OCI compartmentName (stored in project_id) when tags are missing
is_oci = df["source"].eq("oci")
comp = df["project_id"].fillna("").astype(str)
fill_alloc("tag_owner", comp.map(oci_owner).where(is_oci), "inferred_oci")
fill_alloc("tag_env", comp.map(oci_env).where(is_oci), "inferred_oci")

# Infer from SaaS vendor when tags are missing
is_saas = df["source"].eq("saas")
vendor = df["account_id"].fillna("").astype(str)
fill_alloc("tag_owner", vendor.map(saas_owner).where(is_saas), "inferred_saas")
fill_alloc("tag_app", vendor.map(saas_app).where(is_saas), "inferred_saas")
fill_alloc("tag_env", np.where(is_saas, "prod", None), "inferred_saas")
fill_alloc("tag_cost_center", np.where(is_saas, "CC300-Tech", None), "inferred_saas")
fill_alloc("tag_business_unit", np.where(is_saas, "UEN-C", None), "inferred_saas")

# Mark remaining gaps explicitly as unallocated for transparent reporting
for c in required:
    df[c] = df[c].fillna("unallocated")

# Compute compliance and cost-weighted coverage
df["is_tag_compliant"] = df[required].ne("unallocated").all(axis=1)
df["net_cost"] = pd.to_numeric(df["net_cost"], errors="coerce").fillna(0.0)

coverage = (df.assign(allocated_cost=np.where(df["is_tag_compliant"], df["net_cost"], 0.0))
              .groupby("source", as_index=False)[["net_cost","allocated_cost"]].sum())
coverage["coverage_pct"] = np.where(coverage["net_cost"] > 0, (coverage["allocated_cost"]/coverage["net_cost"])*100, 0).round(2)

# Save allocation-ready dataset for KPI/anomaly/forecast steps
df.to_csv(OUT / "finops_allocated_line_items.csv", index=False)

print("Saved:", OUT / "finops_allocated_line_items.csv")
print(coverage.sort_values("coverage_pct", ascending=False).to_string(index=False))
df.head(5)


Saved: /content/finops_showcase/output/finops_allocated_line_items.csv
source    net_cost  allocated_cost  coverage_pct
   k8s    215.4104        215.4104        100.00
   gcp   5194.3675       5194.3675        100.00
  saas 152805.5202     152805.5202        100.00
   oci   4877.4601       2846.9745         58.37
 azure   6789.5818       2709.9621         39.91
   aws   9183.0946       3642.3017         39.66


,date,source,account_id,project_id,service,sku,region,resource_id,usage_amount,usage_unit,cost,net_cost,amortized_cost,commitment_type,tag_owner,tag_app,tag_env,tag_cost_center,tag_business_unit,raw_tags_json,alloc_method,is_tag_compliant
0,2025-11-20,aws,176372599199,NaN,Storage,GB-Mo,eastus,arn:aws:res/167554617,7.0296,GB-Mo,6.3751,6.3751,6.3751,NaN,claudio,mobile-app,test,CC400-Finance,UEN-B,"{""owner"": ""claudio"", ""app"": ""mobile-app"", ""env...",tagged,True
1,2025-12-11,aws,560378022973,NaN,AI/ML,DataTransfer,sa-east-1,arn:aws:res/551386820,2.3991,Requests,2.6676,2.6676,2.6676,NaN,diego,erp-integration,test,CC400-Finance,UEN-B,"{""owner"": ""diego"", ""app"": ""erp-integration"", ""...",tagged,True
2,2025-12-07,aws,505663829036,NaN,Storage,DataTransfer,us-east-1,arn:aws:res/612920837,0.3539,GB,0.3210,0.3210,0.3210,NaN,diego,ai-agent-vt,unallocated,CC100-Sales,UEN-B,"{""owner"": ""diego"", ""app"": ""ai-agent-vt"", ""cost...",inferred_k8s,False
3,2025-12-01,aws,776180930142,NaN,Storage,DataTransfer,sa-east-1,arn:aws:res/759155889,3.7181,GB,3.2768,3.2768,3.2768,NaN,ana,ai-agent-vt,dev,unallocated,UEN-A,"{""owner"": ""ana"", ""app"": ""ai-agent-vt"", ""env"": ...",inferred_k8s,False
4,2025-11-30,aws,518142328473,NaN,Database,Requests,eastus,arn:aws:res/269402834,3.1550,Hrs,0.9844,0.9844,0.9844,NaN,diego,sales-portal,test,CC300-Tech,UEN-C,"{""owner"": ""diego"", ""app"": ""sales-portal"", ""env...",tagged,True


In [ ]:
# Load allocation-ready line items, compute core FinOps KPIs (total/net/amortized, allocation health, showback/chargeback views, top drivers, daily trend), and save summary tables.
import pandas as pd
import numpy as np
import json
from pathlib import Path

BASE = Path("/content/finops_showcase")
OUT  = BASE / "output"

df = pd.read_csv(OUT / "finops_allocated_line_items.csv", parse_dates=["date"])

# Ensure numeric cost fields
for c in ["cost", "net_cost", "amortized_cost"]:
    df[c] = pd.to_numeric(df.get(c, 0), errors="coerce").fillna(0.0)

# Define allocation/compliance and unallocated spend
required = ["tag_owner","tag_app","tag_env","tag_cost_center","tag_business_unit"]
df["is_unallocated"] = df[required].eq("unallocated").any(axis=1)

# Compute headline KPIs
total_cost = float(df["cost"].sum())
total_net  = float(df["net_cost"].sum())
total_amort= float(df["amortized_cost"].sum())
unalloc_net = float(df.loc[df["is_unallocated"], "net_cost"].sum())
alloc_coverage_pct = round(100 * (1 - (unalloc_net / total_net)) if total_net > 0 else 0, 2)

# Compute discount signal where available (positive means credits/discounts reduced net vs cost)
df["discount_amount"] = (df["cost"] - df["net_cost"]).clip(lower=0)
discount_pct = round(100 * (df["discount_amount"].sum() / df["cost"].sum()) if df["cost"].sum() > 0 else 0, 2)

kpis = {
    "rows": int(len(df)),
    "total_cost": round(total_cost, 2),
    "total_net_cost": round(total_net, 2),
    "total_amortized_cost": round(total_amort, 2),
    "unallocated_net_cost": round(unalloc_net, 2),
    "allocation_coverage_pct": alloc_coverage_pct,
    "discount_pct_of_cost": discount_pct,
}

# Build summary tables
by_source = (df.groupby("source", as_index=False)[["net_cost","amortized_cost","cost","discount_amount"]]
               .sum().sort_values("net_cost", ascending=False))
by_owner  = (df.groupby("tag_owner", as_index=False)[["net_cost"]].sum()
               .sort_values("net_cost", ascending=False))
by_app    = (df.groupby(["tag_app","tag_env"], as_index=False)[["net_cost"]].sum()
               .sort_values("net_cost", ascending=False))
showback_cost_center = (df.groupby(["tag_cost_center","tag_business_unit"], as_index=False)[["net_cost"]].sum()
                          .sort_values("net_cost", ascending=False))
chargeback_owner_app = (df.groupby(["tag_owner","tag_app"], as_index=False)[["net_cost"]].sum()
                          .sort_values("net_cost", ascending=False))

top_drivers = (df.groupby(["source","service","sku"], as_index=False)[["net_cost"]].sum()
                 .sort_values("net_cost", ascending=False).head(20))

daily = (df.groupby(["date","source"], as_index=False)[["net_cost"]].sum()
           .sort_values(["date","source"]))
daily_total = df.groupby("date", as_index=False)[["net_cost"]].sum().sort_values("date")
daily_total["net_cost_ma7"] = daily_total["net_cost"].rolling(7, min_periods=1).mean()

# Save outputs
(OUT / "kpis").mkdir(parents=True, exist_ok=True)
with open(OUT / "kpis" / "kpi_overview.json", "w", encoding="utf-8") as f:
    json.dump(kpis, f, indent=2)

by_source.to_csv(OUT / "kpis" / "by_source.csv", index=False)
by_owner.to_csv(OUT / "kpis" / "by_owner.csv", index=False)
by_app.to_csv(OUT / "kpis" / "by_app_env.csv", index=False)
showback_cost_center.to_csv(OUT / "kpis" / "showback_cost_center_bu.csv", index=False)
chargeback_owner_app.to_csv(OUT / "kpis" / "chargeback_owner_app.csv", index=False)
top_drivers.to_csv(OUT / "kpis" / "top_cost_drivers.csv", index=False)
daily.to_csv(OUT / "kpis" / "daily_by_source.csv", index=False)
daily_total.to_csv(OUT / "kpis" / "daily_total_ma7.csv", index=False)

print("KPI overview:", kpis)
print("\nTop sources by net cost:\n", by_source.head(6).to_string(index=False))
print("\nTop 10 owners by net cost:\n", by_owner.head(10).to_string(index=False))


KPI overview: {'rows': 10380, 'total_cost': 180235.6, 'total_net_cost': 179065.43, 'total_amortized_cost': 178268.22, 'unallocated_net_cost': 11650.9, 'allocation_coverage_pct': 93.49, 'discount_pct_of_cost': np.float64(0.65)}

Top sources by net cost:
 source    net_cost  amortized_cost        cost  discount_amount
  saas 152805.5202     152805.5202 152805.5202           0.0000
   aws   9183.0946       8385.8796   9183.0946           0.0000
 azure   6789.5818       6789.5818   7300.1634         510.5816
   gcp   5194.3675       5194.3675   5631.6468         437.2793
   oci   4877.4601       4877.4601   5099.7623         222.3022
   k8s    215.4104        215.4104    215.4104           0.0000

Top 10 owners by net cost:
   tag_owner   net_cost
    claudio 60528.9789
      diego 34256.3751
      pedro 27953.5894
  sebastian 27258.2131
        ana 26284.5986
unallocated  2783.6795


In [ ]:
# Detect daily cost anomalies (spikes/drops) using a robust z-score (MAD), then save an “alerts” table for review.
import pandas as pd
import numpy as np
from pathlib import Path

BASE = Path("/content/finops_showcase")
OUT  = BASE / "output"
ANOM = OUT / "anomalies"
ANOM.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(OUT / "finops_allocated_line_items.csv", parse_dates=["date"])
df["net_cost"] = pd.to_numeric(df["net_cost"], errors="coerce").fillna(0.0)

# Aggregate daily spend by source + app (good granularity for early detection)
daily = (df.groupby(["date","source","tag_app"], as_index=False)["net_cost"].sum()
           .sort_values(["source","tag_app","date"]))

# Compute robust z-score per group (median + MAD)
def robust_z(s: pd.Series) -> pd.Series:
    med = s.median()
    mad = (s - med).abs().median()
    denom = 1.4826 * mad if mad and mad > 0 else np.nan
    return (s - med) / denom

daily["rz"] = daily.groupby(["source","tag_app"])["net_cost"].transform(robust_z)

# Flag anomalies (tune threshold as desired)
TH = 3.5
daily["anomaly_flag"] = daily["rz"].abs() >= TH

alerts = (daily[daily["anomaly_flag"]]
          .copy()
          .sort_values("rz", key=lambda x: x.abs(), ascending=False))

# Add lightweight context: previous day and 7-day rolling mean for the same group
daily["prev_day_cost"] = daily.groupby(["source","tag_app"])["net_cost"].shift(1)
daily["ma7"] = daily.groupby(["source","tag_app"])["net_cost"].transform(lambda s: s.rolling(7, min_periods=3).mean())
alerts = alerts.merge(daily[["date","source","tag_app","prev_day_cost","ma7"]], on=["date","source","tag_app"], how="left")

# Save alerts
alerts.to_csv(ANOM / "daily_cost_anomalies.csv", index=False)

print("Anomaly alerts saved:", ANOM / "daily_cost_anomalies.csv")
print("Alerts found:", len(alerts))
alerts.head(15)


Anomaly alerts saved: /content/finops_showcase/output/anomalies/daily_cost_anomalies.csv
Alerts found: 4


,date,source,tag_app,net_cost,rz,anomaly_flag,prev_day_cost,ma7
0,2025-11-25,gcp,erp-integration,76.5333,5.033272,True,26.8956,39.570957
1,2025-12-13,aws,ai-agent-vt,124.3655,4.823416,True,39.2343,63.069471
2,2025-11-27,gcp,erp-integration,69.3699,4.263736,True,26.2498,41.283386
3,2025-12-14,azure,data-platform,83.5366,3.599011,True,35.5666,42.683000


In [ ]:
# Generate optimization candidates (waste + governance + rate checks), estimate potential savings, and save a prioritized opportunities table.
import pandas as pd
import numpy as np
from pathlib import Path

BASE = Path("/content/finops_showcase")
OUT  = BASE / "output"
OPT  = OUT / "optimizations"
OPT.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(OUT / "finops_allocated_line_items.csv", parse_dates=["date"])
for c in ["cost","net_cost","amortized_cost","usage_amount"]:
    df[c] = pd.to_numeric(df.get(c, 0), errors="coerce").fillna(0.0)

required = ["tag_owner","tag_app","tag_env","tag_cost_center","tag_business_unit"]
df["has_unallocated_tag"] = df[required].eq("unallocated").any(axis=1)

opps = []

# Rule 1: Detect “always-on” non-prod spend (schedule/stop-start opportunity)
nonprod = df[df["tag_env"].isin(["dev","test"])].copy()
nonprod["key"] = nonprod["resource_id"].fillna(nonprod["sku"].fillna("unknown"))
g = (nonprod.groupby(["source","tag_app","tag_env","service","key"], as_index=False)
          .agg(active_days=("date","nunique"), baseline_net_cost=("net_cost","sum"),
               owner=("tag_owner","first"), cc=("tag_cost_center","first"), bu=("tag_business_unit","first")))
cand = g[(g["active_days"] >= 26) & (g["baseline_net_cost"] > 20)]
for _, r in cand.iterrows():
    opps.append({
        "opportunity_type": "schedule_nonprod_always_on",
        "severity": "high" if r["baseline_net_cost"] > 200 else "medium",
        "source": r["source"],
        "tag_owner": r["owner"],
        "tag_app": r["tag_app"],
        "tag_env": r["tag_env"],
        "tag_cost_center": r["cc"],
        "tag_business_unit": r["bu"],
        "service": r["service"],
        "sku": None,
        "resource_id": r["key"],
        "baseline_net_cost_period": round(r["baseline_net_cost"], 2),
        "suggested_action": "Apply start/stop schedules for non-prod (nights/weekends) and enforce TTL for short-lived environments.",
        "assumed_savings_pct": 0.40,
        "estimated_savings_period": round(r["baseline_net_cost"] * 0.40, 2),
    })

# Rule 2: Flag unallocated spend (governance improvement; savings shown as 0 but tracked)
unalloc = (df[df["has_unallocated_tag"]]
           .groupby(["source","service"], as_index=False)["net_cost"].sum()
           .sort_values("net_cost", ascending=False))
for _, r in unalloc.head(15).iterrows():
    opps.append({
        "opportunity_type": "tagging_allocation_gap",
        "severity": "high" if r["net_cost"] > 200 else "medium" if r["net_cost"] > 80 else "low",
        "source": r["source"],
        "tag_owner": "unallocated",
        "tag_app": "unallocated",
        "tag_env": "unallocated",
        "tag_cost_center": "unallocated",
        "tag_business_unit": "unallocated",
        "service": r["service"],
        "sku": None,
        "resource_id": None,
        "baseline_net_cost_period": round(r["net_cost"], 2),
        "suggested_action": "Enforce tag policy (required keys), add tag validation at provisioning, and remediate existing untagged resources.",
        "assumed_savings_pct": 0.00,
        "estimated_savings_period": 0.00,
    })

# Rule 3: Identify unusually high effective rates (rate optimization / pricing review)
rate_df = df[(df["usage_amount"] > 0) & (df["net_cost"] > 0)].copy()
rate_df["effective_rate"] = rate_df["net_cost"] / rate_df["usage_amount"]
med = (rate_df.groupby(["source","service","sku"], as_index=False)["effective_rate"]
              .median().rename(columns={"effective_rate":"median_rate"}))
rate_df = rate_df.merge(med, on=["source","service","sku"], how="left")
rate_df["rate_ratio"] = np.where(rate_df["median_rate"] > 0, rate_df["effective_rate"]/rate_df["median_rate"], np.nan)

thr_cost = rate_df["net_cost"].quantile(0.90) if len(rate_df) else 0
high_rate = rate_df[(rate_df["rate_ratio"] >= 2.0) & (rate_df["net_cost"] >= thr_cost)].copy()

grp = (high_rate.groupby(["source","service","sku","region","tag_app","tag_env"], as_index=False)
              .agg(baseline_net_cost=("net_cost","sum"),
                   owner=("tag_owner","first"), cc=("tag_cost_center","first"), bu=("tag_business_unit","first")))
for _, r in grp.head(20).iterrows():
    opps.append({
        "opportunity_type": "high_effective_rate_review",
        "severity": "high" if r["baseline_net_cost"] > 150 else "medium",
        "source": r["source"],
        "tag_owner": r["owner"],
        "tag_app": r["tag_app"],
        "tag_env": r["tag_env"],
        "tag_cost_center": r["cc"],
        "tag_business_unit": r["bu"],
        "service": r["service"],
        "sku": r["sku"],
        "resource_id": r["region"],
        "baseline_net_cost_period": round(r["baseline_net_cost"], 2),
        "suggested_action": "Review pricing: region/SKU selection, tiering, commitments, and discounts; validate that usage units match expectations.",
        "assumed_savings_pct": 0.15,
        "estimated_savings_period": round(r["baseline_net_cost"] * 0.15, 2),
    })

# Rule 4: Kubernetes hot spots (requests/limits + autoscaling + bin packing)
k8s = df[df["source"].eq("k8s")].copy()
k8s_grp = (k8s.groupby(["project_id","tag_app","tag_env"], as_index=False)["net_cost"].sum()
             .sort_values("net_cost", ascending=False).head(10))
for _, r in k8s_grp.iterrows():
    opps.append({
        "opportunity_type": "k8s_namespace_hotspot",
        "severity": "high" if r["net_cost"] > 200 else "medium" if r["net_cost"] > 80 else "low",
        "source": "k8s",
        "tag_owner": "mixed",
        "tag_app": r["tag_app"],
        "tag_env": r["tag_env"],
        "tag_cost_center": "mixed",
        "tag_business_unit": "mixed",
        "service": "Kubernetes",
        "sku": None,
        "resource_id": r["project_id"],
        "baseline_net_cost_period": round(r["net_cost"], 2),
        "suggested_action": "Tune requests/limits, enable autoscaling, remove over-provisioned workloads, and improve bin packing on nodes.",
        "assumed_savings_pct": 0.20,
        "estimated_savings_period": round(r["net_cost"] * 0.20, 2),
    })

# Rule 5: SaaS rationalization candidates (seat/usage review)
saas = df[df["source"].eq("saas")].copy()
saas_grp = (saas.groupby(["account_id","sku"], as_index=False)["net_cost"].sum()
              .sort_values("net_cost", ascending=False).head(12))
for _, r in saas_grp.iterrows():
    opps.append({
        "opportunity_type": "saas_rationalization",
        "severity": "high" if r["net_cost"] > 200 else "medium" if r["net_cost"] > 80 else "low",
        "source": "saas",
        "tag_owner": "mixed",
        "tag_app": "mixed",
        "tag_env": "prod",
        "tag_cost_center": "mixed",
        "tag_business_unit": "mixed",
        "service": r["account_id"],
        "sku": r["sku"],
        "resource_id": None,
        "baseline_net_cost_period": round(r["net_cost"], 2),
        "suggested_action": "Review licenses/usage, remove inactive seats, consolidate plans, and apply volume discounts where possible.",
        "assumed_savings_pct": 0.10,
        "estimated_savings_period": round(r["net_cost"] * 0.10, 2),
    })

opps_df = pd.DataFrame(opps)
opps_df["estimated_savings_period"] = pd.to_numeric(opps_df["estimated_savings_period"], errors="coerce").fillna(0.0)
opps_df = opps_df.sort_values(["estimated_savings_period","baseline_net_cost_period"], ascending=False)

opps_df.to_csv(OPT / "optimization_opportunities.csv", index=False)

print("Saved:", OPT / "optimization_opportunities.csv")
print("Opportunities:", len(opps_df), "| Total estimated savings (period):", round(opps_df["estimated_savings_period"].sum(), 2))
opps_df.head(15)


Saved: /content/finops_showcase/output/optimizations/optimization_opportunities.csv
Opportunities: 57 | Total estimated savings (period): 9368.71


,opportunity_type,severity,source,tag_owner,tag_app,tag_env,tag_cost_center,tag_business_unit,service,sku,resource_id,baseline_net_cost_period,suggested_action,assumed_savings_pct,estimated_savings_period
45,saas_rationalization,high,saas,mixed,mixed,prod,mixed,mixed,Slack,APM,None,11704.78,"Review licenses/usage, remove inactive seats, ...",0.10,1170.48
46,saas_rationalization,high,saas,mixed,mixed,prod,mixed,mixed,OpenAI,API Usage,None,10712.40,"Review licenses/usage, remove inactive seats, ...",0.10,1071.24
47,saas_rationalization,high,saas,mixed,mixed,prod,mixed,mixed,Datadog,APM,None,10538.76,"Review licenses/usage, remove inactive seats, ...",0.10,1053.88
48,saas_rationalization,high,saas,mixed,mixed,prod,mixed,mixed,GitHub,API Usage,None,10162.62,"Review licenses/usage, remove inactive seats, ...",0.10,1016.26
49,saas_rationalization,high,saas,mixed,mixed,prod,mixed,mixed,Snowflake,Warehouse,None,8469.37,"Review licenses/usage, remove inactive seats, ...",0.10,846.94
50,saas_rationalization,high,saas,mixed,mixed,prod,mixed,mixed,Datadog,Seats,None,7609.16,"Review licenses/usage, remove inactive seats, ...",0.10,760.92
51,saas_rationalization,high,saas,mixed,mixed,prod,mixed,mixed,Snowflake,Seats,None,6203.01,"Review licenses/usage, remove inactive seats, ...",0.10,620.30
52,saas_rationalization,high,saas,mixed,mixed,prod,mixed,mixed,Datadog,API Usage,None,6033.04,"Review licenses/usage, remove inactive seats, ...",0.10,603.30
53,saas_rationalization,high,saas,mixed,mixed,prod,mixed,mixed,Snowflake,API Usage,None,5926.26,"Review licenses/usage, remove inactive seats, ...",0.10,592.63
54,saas_rationalization,high,saas,mixed,mixed,prod,mixed,mixed,OpenAI,Seats,None,5716.95,"Review licenses/usage, remove inactive seats, ...",0.10,571.70


In [ ]:
# Load allocated dataset, build daily spend series, forecast next 14 days (simple linear trend), define budgets, and save budget/forecast alerts.
import pandas as pd
import numpy as np
from pathlib import Path

BASE = Path("/content/finops_showcase")
OUT  = BASE / "output"
FRC  = OUT / "forecasting"
FRC.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(OUT / "finops_allocated_line_items.csv", parse_dates=["date"])
df["net_cost"] = pd.to_numeric(df["net_cost"], errors="coerce").fillna(0.0)

# Aggregate daily net cost by app and also total
daily_app = (df.groupby(["date","tag_app"], as_index=False)["net_cost"].sum()
               .sort_values(["tag_app","date"]))
daily_total = (df.groupby("date", as_index=False)["net_cost"].sum()
                 .sort_values("date"))

def forecast_linear(daily_df: pd.DataFrame, group_cols, value_col="net_cost", horizon_days=14, min_points=7):
    out = []
    if group_cols:
        groups = daily_df.groupby(group_cols)
    else:
        groups = [((), daily_df)]
    for key, g in groups:
        g = g.sort_values("date").copy()
        g = g[g[value_col].notna()]
        if len(g) == 0:
            continue
        y = g[value_col].to_numpy(dtype=float)
        # Use last window to fit a simple trend
        window = max(min_points, min(len(y), 14))
        y_w = y[-window:]
        x_w = np.arange(window, dtype=float)
        if window >= 2 and np.std(y_w) > 0:
            a, b = np.polyfit(x_w, y_w, 1)  # y = a*x + b
        else:
            a, b = 0.0, float(np.mean(y_w))
        last_date = pd.to_datetime(g["date"].iloc[-1]).normalize()
        future_dates = pd.date_range(last_date + pd.Timedelta(days=1), periods=horizon_days, freq="D")
        x_f = np.arange(window, window + horizon_days, dtype=float)
        y_f = (a * x_f + b)
        y_f = np.clip(y_f, 0, None)
        for d, v in zip(future_dates, y_f):
            row = {"date": d, "forecast_net_cost": float(v)}
            if group_cols:
                if isinstance(key, tuple):
                    for c, kv in zip(group_cols, key):
                        row[c] = kv
                else:
                    row[group_cols[0]] = key
            out.append(row)
    return pd.DataFrame(out)

# Forecast next 14 days by app and total
fc_app = forecast_linear(daily_app, ["tag_app"], horizon_days=14)
fc_total = forecast_linear(daily_total, [], horizon_days=14)

# Define simple budgets per app (baseline = last 30 days net cost; budget = baseline * 1.10)
app_30d = (df.groupby("tag_app", as_index=False)["net_cost"].sum()
             .rename(columns={"net_cost":"baseline_30d_net_cost"}))
app_30d["budget_next_30d"] = (app_30d["baseline_30d_net_cost"] * 1.10).round(2)

# Project next 30 days per app using recent 7-day average daily spend
last_date = df["date"].max().normalize()
start_7 = last_date - pd.Timedelta(days=6)
recent7 = (df[df["date"].between(start_7, last_date)]
           .groupby("tag_app", as_index=False)["net_cost"].sum()
           .rename(columns={"net_cost":"recent_7d_net_cost"}))
recent7["projected_next_30d"] = (recent7["recent_7d_net_cost"] / 7.0 * 30.0).round(2)

budget_check = app_30d.merge(recent7, on="tag_app", how="left").fillna({"recent_7d_net_cost":0.0, "projected_next_30d":0.0})
budget_check["budget_delta"] = (budget_check["projected_next_30d"] - budget_check["budget_next_30d"]).round(2)
budget_check["budget_risk"] = np.where(budget_check["budget_delta"] > 0, "at_risk", "ok")

alerts = budget_check.sort_values(["budget_risk","budget_delta","projected_next_30d"], ascending=[True, False, False])

# Save outputs
daily_app.to_csv(FRC / "daily_by_app.csv", index=False)
daily_total.to_csv(FRC / "daily_total.csv", index=False)
fc_app.to_csv(FRC / "forecast_next_14d_by_app.csv", index=False)
fc_total.to_csv(FRC / "forecast_next_14d_total.csv", index=False)
alerts.to_csv(FRC / "budget_alerts_by_app.csv", index=False)

print("Saved:", FRC)
print("Budget alerts (top 10):")
print(alerts.head(10).to_string(index=False))


Saved: /content/finops_showcase/output/forecasting
Budget alerts (top 10):
        tag_app  baseline_30d_net_cost  budget_next_30d  recent_7d_net_cost  projected_next_30d  budget_delta budget_risk
    unallocated              3412.2737          3753.50            786.2309             3369.56       -383.94          ok
     mobile-app             23985.3942         26383.93           1083.4866             4643.51     -21740.42          ok
    ai-agent-vt             28660.2599         31526.29           1216.5689             5213.87     -26312.42          ok
erp-integration             35847.6236         39432.39            900.2108             3858.05     -35574.34          ok
   sales-portal             36828.1960         40511.02           1077.5062             4617.88     -35893.14          ok
  data-platform             50331.6872         55364.86            938.8999             4023.86     -51341.00          ok


In [ ]:
# Build unit economics (cost per transaction / cost per active user) by app using simulated business metrics, generate simple charts, and save unit-cost tables.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

BASE = Path("/content/finops_showcase")
OUT  = BASE / "output"
UE   = OUT / "unit_economics"
UE.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(OUT / "finops_allocated_line_items.csv", parse_dates=["date"])
df["net_cost"] = pd.to_numeric(df["net_cost"], errors="coerce").fillna(0.0)

daily_app = (df.groupby(["date","tag_app"], as_index=False)["net_cost"].sum()
               .sort_values(["tag_app","date"]))

# Simulate app usage metrics (transactions + active users) with mild weekly seasonality
rng = np.random.default_rng(7)
apps = daily_app["tag_app"].fillna("unallocated").unique().tolist()
dates = pd.to_datetime(daily_app["date"].unique())
dates = pd.Series(dates).sort_values().to_list()

base_txn = {a: float(rng.integers(500, 5000)) for a in apps}
base_usr = {a: float(rng.integers(50, 800)) for a in apps}

rows = []
for d in dates:
    dow = d.weekday()
    season = 1.0 + (0.10 if dow in [1,2,3] else (-0.08 if dow in [5,6] else 0.0))
    noise_txn = rng.normal(1.0, 0.12, len(apps))
    noise_usr = rng.normal(1.0, 0.10, len(apps))
    for i, a in enumerate(apps):
        tx = max(1.0, base_txn[a] * season * noise_txn[i])
        us = max(1.0, base_usr[a] * season * noise_usr[i])
        rows.append({"date": d, "tag_app": a, "transactions": round(tx, 2), "active_users": round(us, 2)})

metrics = pd.DataFrame(rows)
metrics.to_csv(UE / "app_usage_metrics.csv", index=False)

# Join cost with metrics and compute unit costs
u = daily_app.merge(metrics, on=["date","tag_app"], how="left")
u["transactions"] = pd.to_numeric(u["transactions"], errors="coerce").fillna(0.0)
u["active_users"] = pd.to_numeric(u["active_users"], errors="coerce").fillna(0.0)

u["cost_per_txn"] = np.where(u["transactions"] > 0, u["net_cost"] / u["transactions"], np.nan)
u["cost_per_active_user"] = np.where(u["active_users"] > 0, u["net_cost"] / u["active_users"], np.nan)

u.to_csv(UE / "daily_unit_cost_by_app.csv", index=False)

# Summarize last-30-days unit economics by app
sum_app = (u.groupby("tag_app", as_index=False)
             .agg(net_cost_30d=("net_cost","sum"),
                  transactions_30d=("transactions","sum"),
                  active_users_30d=("active_users","sum")))
sum_app["avg_cost_per_txn_30d"] = np.where(sum_app["transactions_30d"] > 0, sum_app["net_cost_30d"]/sum_app["transactions_30d"], np.nan)
sum_app["avg_cost_per_active_user_30d"] = np.where(sum_app["active_users_30d"] > 0, sum_app["net_cost_30d"]/sum_app["active_users_30d"], np.nan)
sum_app = sum_app.sort_values("net_cost_30d", ascending=False)
sum_app.to_csv(UE / "unit_economics_by_app_30d.csv", index=False)

# Plot 1: unit cost trend for top 4 apps by spend
top_apps = sum_app["tag_app"].head(4).to_list()
plot_df = u[u["tag_app"].isin(top_apps)].copy()
plt.figure()
for a in top_apps:
    g = plot_df[plot_df["tag_app"].eq(a)].sort_values("date")
    plt.plot(g["date"], g["cost_per_txn"], label=a)
plt.title("Cost per Transaction (daily) — Top Apps by Spend")
plt.xlabel("Date")
plt.ylabel("Cost per Transaction")
plt.legend()
plt.tight_layout()
plt.savefig(UE / "unit_cost_trend_top_apps.png", dpi=160)
plt.close()

# Plot 2: spend vs transactions (last 30 days) to visualize efficiency
plt.figure()
plt.scatter(sum_app["transactions_30d"], sum_app["net_cost_30d"])
plt.title("Spend vs Transactions (30d) — by App")
plt.xlabel("Transactions (30d)")
plt.ylabel("Net Cost (30d)")
plt.tight_layout()
plt.savefig(UE / "spend_vs_transactions_scatter.png", dpi=160)
plt.close()

print("Saved unit-economics outputs in:", UE)
print("Top 8 apps by net cost (30d):")
print(sum_app.head(8).to_string(index=False))


Saved unit-economics outputs in: /content/finops_showcase/output/unit_economics
Top 8 apps by net cost (30d):
        tag_app  net_cost_30d  transactions_30d  active_users_30d  avg_cost_per_txn_30d  avg_cost_per_active_user_30d
  data-platform    50331.6872         102327.38           6596.85              0.491869                      7.629655
   sales-portal    36828.1960          94455.95           7960.20              0.389898                      4.626542
erp-integration    35847.6236         107008.52           2769.10              0.334998                     12.945587
    ai-agent-vt    28660.2599         144043.93          20793.86              0.198969                      1.378304
     mobile-app    23985.3942         135338.21           8274.52              0.177226                      2.898705
    unallocated     3412.2737         123957.90          21952.22              0.027528                      0.155441


In [ ]:
# Install Plotly (only if missing), aggregate daily spend, and render an interactive line chart (also save as HTML for later dashboard)
try:
    import plotly.graph_objects as go
except Exception:
    !pip -q install plotly
    import plotly.graph_objects as go

import pandas as pd
from pathlib import Path

OUT = Path("/content/finops_showcase/output")
df = pd.read_csv(OUT / "finops_allocated_line_items.csv", parse_dates=["date"])
df["net_cost"] = pd.to_numeric(df["net_cost"], errors="coerce").fillna(0.0)

daily = df.groupby("date", as_index=False)["net_cost"].sum().sort_values("date")
daily["ma7"] = daily["net_cost"].rolling(7, min_periods=1).mean()

fig = go.Figure()
fig.add_trace(go.Scatter(x=daily["date"], y=daily["net_cost"], mode="lines+markers", name="Daily net cost"))
fig.add_trace(go.Scatter(x=daily["date"], y=daily["ma7"], mode="lines", name="MA7"))
fig.update_layout(
    title="Daily Net Cost (Total) + 7-day Moving Average",
    xaxis_title="Date",
    yaxis_title="Net Cost",
    template="plotly_white",
    height=450
)
fig.show()

(OUT / "plots").mkdir(parents=True, exist_ok=True)
fig.write_html(OUT / "plots" / "daily_net_cost_ma7.html", include_plotlyjs="cdn")
print("Saved:", OUT / "plots" / "daily_net_cost_ma7.html")


Saved: /content/finops_showcase/output/plots/daily_net_cost_ma7.html


In [ ]:
# Chart 2: Net cost by source (interactive Plotly bar) + save as HTML
try:
    import plotly.express as px
except Exception:
    !pip -q install plotly
    import plotly.express as px

import pandas as pd
from pathlib import Path

OUT = Path("/content/finops_showcase/output")
df = pd.read_csv(OUT / "finops_allocated_line_items.csv", parse_dates=["date"])
df["net_cost"] = pd.to_numeric(df["net_cost"], errors="coerce").fillna(0.0)

by_source = (df.groupby("source", as_index=False)["net_cost"].sum()
             .sort_values("net_cost", ascending=False))

fig = px.bar(by_source, x="source", y="net_cost", title="Net Cost by Source (Total Period)")
fig.update_layout(template="plotly_white", height=450, xaxis_title="Source", yaxis_title="Net Cost")
fig.show()

(OUT / "plots").mkdir(parents=True, exist_ok=True)
fig.write_html(OUT / "plots" / "net_cost_by_source.html", include_plotlyjs="cdn")
print("Saved:", OUT / "plots" / "net_cost_by_source.html")


Saved: /content/finops_showcase/output/plots/net_cost_by_source.html


In [ ]:
# Chart 3: Top apps by net cost (interactive Plotly bar) + save as HTML
try:
    import plotly.express as px
except Exception:
    !pip -q install plotly
    import plotly.express as px

import pandas as pd
from pathlib import Path

OUT = Path("/content/finops_showcase/output")
df = pd.read_csv(OUT / "finops_allocated_line_items.csv", parse_dates=["date"])
df["net_cost"] = pd.to_numeric(df["net_cost"], errors="coerce").fillna(0.0)

by_app = (df.groupby("tag_app", as_index=False)["net_cost"].sum()
          .sort_values("net_cost", ascending=False)
          .head(12))

fig = px.bar(by_app, x="tag_app", y="net_cost", title="Top Apps by Net Cost (Total Period)")
fig.update_layout(template="plotly_white", height=450, xaxis_title="App", yaxis_title="Net Cost")
fig.show()

(OUT / "plots").mkdir(parents=True, exist_ok=True)
fig.write_html(OUT / "plots" / "top_apps_by_net_cost.html", include_plotlyjs="cdn")
print("Saved:", OUT / "plots" / "top_apps_by_net_cost.html")


Saved: /content/finops_showcase/output/plots/top_apps_by_net_cost.html


In [ ]:
# Chart 4: Anomaly alerts (interactive Plotly scatter: rz-score vs net cost) + save as HTML
try:
    import plotly.express as px
except Exception:
    !pip -q install plotly
    import plotly.express as px

import pandas as pd
from pathlib import Path

OUT = Path("/content/finops_showcase/output")
anoms_path = OUT / "anomalies" / "daily_cost_anomalies.csv"
anoms = pd.read_csv(anoms_path, parse_dates=["date"])

# Keep top 40 strongest anomalies (by absolute rz)
if "rz" in anoms.columns:
    anoms = anoms.sort_values("rz", key=lambda s: s.abs(), ascending=False).head(40)

# Make sure numeric fields are numeric
for c in ["net_cost","rz","prev_day_cost","ma7"]:
    if c in anoms.columns:
        anoms[c] = pd.to_numeric(anoms[c], errors="coerce")

fig = px.scatter(
    anoms,
    x="date",
    y="net_cost",
    color="source" if "source" in anoms.columns else None,
    size=anoms["rz"].abs() if "rz" in anoms.columns else None,
    hover_data=[c for c in ["tag_app","rz","prev_day_cost","ma7"] if c in anoms.columns],
    title="Anomaly Alerts (Top 40): Daily Net Cost Spikes/Drops"
)
fig.update_layout(template="plotly_white", height=450, xaxis_title="Date", yaxis_title="Net Cost")
fig.show()

(OUT / "plots").mkdir(parents=True, exist_ok=True)
fig.write_html(OUT / "plots" / "anomaly_alerts_scatter.html", include_plotlyjs="cdn")
print("Saved:", OUT / "plots" / "anomaly_alerts_scatter.html")


Saved: /content/finops_showcase/output/plots/anomaly_alerts_scatter.html


In [ ]:
# Plot savings by type with clear labels: bar = savings, text = "N=items"
import pandas as pd
from pathlib import Path

try:
    import plotly.express as px
except Exception:
    !pip -q install plotly
    import plotly.express as px

OUT = Path("/content/finops_showcase/output")
opps = pd.read_csv(OUT / "optimizations" / "optimization_opportunities.csv")

opps["estimated_savings_period"] = pd.to_numeric(opps.get("estimated_savings_period", 0), errors="coerce").fillna(0.0)
opps["baseline_net_cost_period"] = pd.to_numeric(opps.get("baseline_net_cost_period", 0), errors="coerce").fillna(0.0)

use = opps[opps["estimated_savings_period"] > 0].copy()

summary = (use.groupby("opportunity_type", as_index=False)
           .agg(estimated_savings_period=("estimated_savings_period","sum"),
                baseline_net_cost_period=("baseline_net_cost_period","sum"),
                items=("opportunity_type","size"))
           .sort_values("estimated_savings_period", ascending=True))

summary["label_items"] = summary["items"].map(lambda n: f"N={int(n)}")
summary["savings_k"] = summary["estimated_savings_period"] / 1000.0  # nicer axis for big numbers

fig = px.bar(
    summary,
    x="savings_k",
    y="opportunity_type",
    orientation="h",
    text="label_items",
    hover_data={"estimated_savings_period":":.2f","baseline_net_cost_period":":.2f","items":True,"savings_k":False},
    title="Optimization Savings by Type — bar = savings, label = number of opportunities"
)
fig.update_layout(template="plotly_white", height=420, xaxis_title="Estimated Savings (k)", yaxis_title="")
fig.update_traces(textposition="outside")
fig.show()

(OUT / "plots").mkdir(parents=True, exist_ok=True)
fig.write_html(OUT / "plots" / "optimization_savings_by_type_v2.html", include_plotlyjs="cdn")
print("Saved:", OUT / "plots" / "optimization_savings_by_type_v2.html")


Saved: /content/finops_showcase/output/plots/optimization_savings_by_type_v2.html


In [ ]:
# Build one clean Plotly dashboard page (5 charts + KPI cards), save HTML, and render inline.
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
from IPython.display import HTML, display

# Plotly
try:
    import plotly.express as px
    import plotly.graph_objects as go
    import plotly.io as pio
except Exception:
    !pip -q install plotly
    import plotly.express as px
    import plotly.graph_objects as go
    import plotly.io as pio

OUT = Path("/content/finops_showcase/output")
PLOTS = OUT / "plots"
PLOTS.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(OUT / "finops_allocated_line_items.csv", parse_dates=["date"])
for c in ["cost","net_cost","amortized_cost","usage_amount"]:
    df[c] = pd.to_numeric(df.get(c, 0), errors="coerce").fillna(0.0)

required = ["tag_owner","tag_app","tag_env","tag_cost_center","tag_business_unit"]
df["is_unallocated"] = df[required].eq("unallocated").any(axis=1)

# KPI cards
total_cost = float(df["cost"].sum())
total_net  = float(df["net_cost"].sum())
unalloc_net = float(df.loc[df["is_unallocated"], "net_cost"].sum())
alloc_cov = 100 * (1 - unalloc_net/total_net) if total_net > 0 else 0.0
discount_amt = float(np.clip(df["cost"] - df["net_cost"], 0, None).sum())
discount_pct = 100 * (discount_amt/total_cost) if total_cost > 0 else 0.0

kpis = [
    ("Total net cost", f"{total_net:,.2f}"),
    ("Allocation coverage", f"{alloc_cov:.2f}%"),
    ("Unallocated net cost", f"{unalloc_net:,.2f}"),
    ("Discount/credits", f"{discount_amt:,.2f} ({discount_pct:.2f}%)"),
    ("Line items", f"{len(df):,}"),
]

# 1) Daily net cost + MA7
daily = df.groupby("date", as_index=False)["net_cost"].sum().sort_values("date")
daily["ma7"] = daily["net_cost"].rolling(7, min_periods=1).mean()
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=daily["date"], y=daily["net_cost"], mode="lines+markers", name="Daily net cost"))
fig1.add_trace(go.Scatter(x=daily["date"], y=daily["ma7"], mode="lines", name="MA7"))
fig1.update_layout(title="Daily Net Cost (Total) + 7-day Moving Average", template="plotly_white", height=420,
                   xaxis_title="Date", yaxis_title="Net Cost")

# 2) Net cost by source
by_source = (df.groupby("source", as_index=False)["net_cost"].sum()
             .sort_values("net_cost", ascending=False))
fig2 = px.bar(by_source, x="source", y="net_cost", title="Net Cost by Source (Total Period)")
fig2.update_layout(template="plotly_white", height=420, xaxis_title="Source", yaxis_title="Net Cost")

# 3) Top apps by net cost
by_app = (df.groupby("tag_app", as_index=False)["net_cost"].sum()
          .sort_values("net_cost", ascending=False).head(12))
fig3 = px.bar(by_app, x="tag_app", y="net_cost", title="Top Apps by Net Cost (Total Period)")
fig3.update_layout(template="plotly_white", height=420, xaxis_title="App", yaxis_title="Net Cost")

# 4) Anomalies scatter (use existing file if present; else compute quickly)
anoms_path = OUT / "anomalies" / "daily_cost_anomalies.csv"
if anoms_path.exists():
    anoms = pd.read_csv(anoms_path, parse_dates=["date"])
else:
    # Aggregate daily spend by source + app and compute robust z-score (MAD)
    d = (df.groupby(["date","source","tag_app"], as_index=False)["net_cost"].sum()
         .sort_values(["source","tag_app","date"]))
    def robust_z(s):
        med = s.median()
        mad = (s - med).abs().median()
        denom = 1.4826 * mad if mad and mad > 0 else np.nan
        return (s - med) / denom
    d["rz"] = d.groupby(["source","tag_app"])["net_cost"].transform(robust_z)
    d["prev_day_cost"] = d.groupby(["source","tag_app"])["net_cost"].shift(1)
    d["ma7"] = d.groupby(["source","tag_app"])["net_cost"].transform(lambda s: s.rolling(7, min_periods=3).mean())
    anoms = d[d["rz"].abs() >= 3.5].copy()

# keep top 40 by |rz|
if not anoms.empty and "rz" in anoms.columns:
    anoms = anoms.sort_values("rz", key=lambda s: s.abs(), ascending=False).head(40)
for c in ["net_cost","rz","prev_day_cost","ma7"]:
    if c in anoms.columns:
        anoms[c] = pd.to_numeric(anoms[c], errors="coerce")

fig4 = px.scatter(
    anoms,
    x="date", y="net_cost",
    color="source" if "source" in anoms.columns else None,
    size=anoms["rz"].abs().fillna(1) if "rz" in anoms.columns and not anoms.empty else None,
    hover_data=[c for c in ["tag_app","rz","prev_day_cost","ma7"] if c in anoms.columns],
    title="Anomaly Alerts (Top 40): Daily Net Cost Spikes/Drops"
)
fig4.update_layout(template="plotly_white", height=420, xaxis_title="Date", yaxis_title="Net Cost")

# 5) Optimization savings by type (clear “N=…” labels)
opps_path = OUT / "optimizations" / "optimization_opportunities.csv"
if opps_path.exists():
    opps = pd.read_csv(opps_path)
    opps["estimated_savings_period"] = pd.to_numeric(opps.get("estimated_savings_period", 0), errors="coerce").fillna(0.0)
    opps["baseline_net_cost_period"] = pd.to_numeric(opps.get("baseline_net_cost_period", 0), errors="coerce").fillna(0.0)
    use = opps[opps["estimated_savings_period"] > 0].copy()
    summary = (use.groupby("opportunity_type", as_index=False)
               .agg(estimated_savings_period=("estimated_savings_period","sum"),
                    baseline_net_cost_period=("baseline_net_cost_period","sum"),
                    items=("opportunity_type","size"))
               .sort_values("estimated_savings_period", ascending=True))
else:
    summary = pd.DataFrame(columns=["opportunity_type","estimated_savings_period","baseline_net_cost_period","items"])

if summary.empty:
    fig5 = go.Figure()
    fig5.update_layout(title="Optimization Savings by Type (Estimated) — no data", template="plotly_white", height=420)
else:
    summary["label_items"] = summary["items"].map(lambda n: f"N={int(n)}")
    summary["savings_k"] = summary["estimated_savings_period"] / 1000.0
    fig5 = px.bar(
        summary,
        x="savings_k", y="opportunity_type",
        orientation="h", text="label_items",
        hover_data={"estimated_savings_period":":.2f","baseline_net_cost_period":":.2f","items":True,"savings_k":False},
        title="Optimization Savings by Type — bar = savings, label = number of opportunities"
    )
    fig5.update_layout(template="plotly_white", height=420, xaxis_title="Estimated Savings (k)", yaxis_title="")
    fig5.update_traces(textposition="outside")

# Convert figures to HTML snippets (include Plotly JS only once)
div1 = pio.to_html(fig1, full_html=False, include_plotlyjs="cdn")
div2 = pio.to_html(fig2, full_html=False, include_plotlyjs=False)
div3 = pio.to_html(fig3, full_html=False, include_plotlyjs=False)
div4 = pio.to_html(fig4, full_html=False, include_plotlyjs=False)
div5 = pio.to_html(fig5, full_html=False, include_plotlyjs=False)

kpi_cards = "".join([f"<div class='kpi'><div class='k'>{k}</div><div class='v'>{v}</div></div>" for k,v in kpis])

css = """
<style>
:root { --bg:#f6f7fb; --card:#fff; --border:#e7e7ef; --muted:#5b6472; --text:#111827; }
body { font-family: ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, Arial; background:var(--bg); color:var(--text); margin:0; }
.container { max-width: 1250px; margin: 0 auto; padding: 18px; }
h1 { margin:0; font-size: 22px; }
.sub { margin:6px 0 14px; color: var(--muted); font-size: 13px; }
.card { background: var(--card); border: 1px solid var(--border); border-radius: 14px; padding: 14px; box-shadow: 0 1px 0 rgba(0,0,0,.03); }
.kpis { display:grid; grid-template-columns: repeat(auto-fit, minmax(220px, 1fr)); gap: 10px; }
.kpi { border: 1px solid var(--border); border-radius: 12px; padding: 10px; background: #fbfbfe; }
.kpi .k { font-size: 12px; color: var(--muted); margin-bottom: 6px; }
.kpi .v { font-size: 18px; font-weight: 700; }
.grid { display:grid; grid-template-columns: 1fr 1fr; gap: 14px; }
@media (max-width: 980px) { .grid { grid-template-columns: 1fr; } }
.plot { border: 1px solid var(--border); border-radius: 12px; padding: 6px; background:#fff; }
</style>
"""

html = f"""
<html><head><meta charset="utf-8">{css}</head>
<body>
  <div class="container">
    <h1>FinOps Showcase Dashboard</h1>
    <div class="sub">Generated {datetime.now().strftime("%Y-%m-%d %H:%M:%S")} — saved to {PLOTS}</div>

    <div class="card"><div class="kpis">{kpi_cards}</div></div>

    <div class="grid" style="margin-top:14px;">
      <div class="card plot">{div1}</div>
      <div class="card plot">{div2}</div>
    </div>

    <div class="grid" style="margin-top:14px;">
      <div class="card plot">{div3}</div>
      <div class="card plot">{div4}</div>
    </div>

    <div class="card plot" style="margin-top:14px;">{div5}</div>
  </div>
</body></html>
"""

dash_path = PLOTS / "finops_dashboard_plotly.html"
dash_path.write_text(html, encoding="utf-8")

display(HTML(html))
print("Saved dashboard:", dash_path)


Saved dashboard: /content/finops_showcase/output/plots/finops_dashboard_plotly.html
